In [12]:
from pprint import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,roc_auc_score,classification_report

# US02 - Experimentação Simples

Esta US busca averiguar a viabilidade da execução da solução de ML para detecção do nível de saúde fetal. Neste momento, não será investido muito esforço em limpeza de dados e seleção de atributos.

Tarefas:
- Implementar uma classe que fará a interface de save e load do dataset (DataAccessHandler).
- Separar conjuntos de treino e teste.
- Salvar conjuntos de treino e teste separadamente.
- Treinar e validar um modelo de Regressão Logística.
- Treinar e validar um modelo de Floresta Aleatória.

Definição de Pronto:
- Ter um modelo V0 que servirá de baseline para comparar com os próximos experimentos. 
- Ter métricas calculadas para o conjunto de treino e validação**.

** O conjunto de validação poderá mudar futuramente, caso se decida executar um processo de validação cruzada. O conjunto de testes se manterá o mesmo, porém ainda não será avaliado aqui para não enviesar futuros resultados.


In [20]:
from data_access_handler import DataAccessHandler

In [22]:
data_handler = DataAccessHandler(main_path="./")
df = data_handler.load(dataset_type="")

# Esta etapa do split só ocorre uma vez no projeto. 
# O time de ciência de dados decidiu fixar o random_state e por consequência 
# o conjunto de testes, por questões de reprodutibilidade.
df_train, df_test = train_test_split(df,train_size=1500,random_state=42)

data_handler.save(df=df_train,dataset_type="train")
data_handler.save(df=df_test,dataset_type="test")

del df_train,df_test,df

In [25]:
df_train = data_handler.load(dataset_type="train")

# Está é uma separação simples entre treino e validação feita apenas para 
# esta primeira etapa de experimentação simples.
# Posteriormente haverá uma classe mais estruturada para lidar com esses 
# splits para experimentação e escolha do melhor modelo.

target = ['fetal_health']
features = df_train.drop(columns=target).columns
X,y = df_train[features],df_train[target].values.ravel()

X_train, X_validation = X[:1100],X[1100:]
y_train,y_validation = y[:1100],y[1100:]

In [26]:
model = LogisticRegression(tol=0.001,max_iter=10000)
model.fit(X_train,y_train)

/Users/lucasthimoteo/miniconda3/envs/base_ml/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=10000, tol=0.001)

In [27]:
y_pred = model.predict(X_train)
pprint(classification_report(y_pred,y_train))

('              precision    recall  f1-score   support\n'
 '\n'
 '         1.0       0.97      0.94      0.95       882\n'
 '         2.0       0.57      0.72      0.63       119\n'
 '         3.0       0.84      0.83      0.83        99\n'
 '\n'
 '    accuracy                           0.90      1100\n'
 '   macro avg       0.79      0.83      0.81      1100\n'
 'weighted avg       0.91      0.90      0.91      1100\n')


In [28]:
y_pred = model.predict(X_validation)
pprint(classification_report(y_pred,y_validation))

('              precision    recall  f1-score   support\n'
 '\n'
 '         1.0       0.96      0.95      0.95       324\n'
 '         2.0       0.52      0.55      0.53        42\n'
 '         3.0       0.73      0.79      0.76        34\n'
 '\n'
 '    accuracy                           0.89       400\n'
 '   macro avg       0.74      0.76      0.75       400\n'
 'weighted avg       0.90      0.89      0.89       400\n')


In [29]:
model = RandomForestClassifier(n_estimators=20,max_depth=3,min_samples_leaf=3)
model.fit(X_train,y_train)

RandomForestClassifier(max_depth=3, min_samples_leaf=3, n_estimators=20)

In [30]:
y_pred = model.predict(X_train)
pprint(classification_report(y_pred,y_train))

('              precision    recall  f1-score   support\n'
 '\n'
 '         1.0       0.99      0.91      0.95       933\n'
 '         2.0       0.51      0.82      0.63        95\n'
 '         3.0       0.70      0.96      0.81        72\n'
 '\n'
 '    accuracy                           0.90      1100\n'
 '   macro avg       0.74      0.90      0.80      1100\n'
 'weighted avg       0.93      0.90      0.91      1100\n')


In [31]:
y_pred = model.predict(X_validation)
pprint(classification_report(y_pred,y_validation))

('              precision    recall  f1-score   support\n'
 '\n'
 '         1.0       0.99      0.92      0.96       344\n'
 '         2.0       0.52      0.77      0.62        30\n'
 '         3.0       0.65      0.92      0.76        26\n'
 '\n'
 '    accuracy                           0.91       400\n'
 '   macro avg       0.72      0.87      0.78       400\n'
 'weighted avg       0.94      0.91      0.92       400\n')


### Conclusões

- Após esta etapa de experimentação muito simples, confirma-se que é possível atingir um nível razoável de performance de classificação.
- Todos os F1 (weighted avg) estão acima de 0.8.
- Analisando algumas métricas no conjunto de validação e treino, conclui-se que o problema é factível de se resolver com modelos de machine learning simples. 